In [1]:
!pip install kaggle


In [2]:
# upload kaggle.json api key from kaggle by going to settings > generate api key to /root/.kaggle folder by going up one level in the file directory and find it
# if the .kaggle folder is not in root then make it
!chmod 600 /root/.kaggle/kaggle.json

In [4]:

import numpy as np
import pandas as pd
import csv
import os


# accuracy score list to keep track of each submissions score on kaggle to be used for the formula on slide 15
accuracy = []
# corresponding filename for accuracy scores
files = []
# holds the prediction vectors from each submission
S = []
STx = []
ratings = []
num_prediction_vectors = 0

# make a few folder under home directory named ensemble_submissions and upload the corresponding files with accuracy names in the form kaggle_output_836.csv
directory = os.fsencode("/home/ensemble_submissions")

set_ratings = 0

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename[-4:] == ".csv":
      num_prediction_vectors += 1
      files.append(filename)
      accuracy_score = int(filename[-7:][:3])/1000
      accuracy.append(accuracy_score)
      df = pd.read_csv("/home/ensemble_submissions/" +filename)
      S.append(np.array(df["Predictor"]))
      N = len(df["Predictor"]) # should be 120000
      Pi = accuracy_score
      stx = 2*Pi-1
      STx.append(stx)
      if set_ratings == 0: # only do once
        ratings = df
        set_ratings +=1

for i in range(len(S)):
    for j in range(len(S[0])):
        if S[i][j] == 0:
            S[i][j] = -1

S = np.array(S)
STx = np.array(STx)

esh = np.matmul(S, np.transpose(S))
iesh = np.linalg.inv(esh)
sesh = np.matmul(np.transpose(S), iesh)
Sensemble = np.matmul(sesh, STx)


for i in range(len(Sensemble)):
    if (i+1) % 6 == 0: # 0-5 hold ratings for first user, do matrix calculations per user to make memory load smaller then add predictios to Sensemble
        S_sub_ensemble = Sensemble[i-5:i+1]
        S_sub_ensemble_sort_max = list(reversed(sorted(S_sub_ensemble)))[0:3]
        index = 0
        for j in range(i-5, i):
            if S_sub_ensemble[index] in S_sub_ensemble_sort_max:
                ratings.at[j, "Predictor"] = 1
            else:
                ratings.at[j, "Predictor"] = 0
            index += 1


ratings.to_csv("kaggle_submission_ensemble.csv", index=False)



!kaggle competitions submit -c aai627-spring2024 -f kaggle_submission_ensemble.csv -m "Ensemble Method num_prediction_vectors = $num_prediction_vectors"
!kaggle competitions submissions -c aai627-spring2024




100% 1.79M/1.79M [00:00<00:00, 3.89MB/s]
Successfully submitted to Music RecommenderWarning: Looks like you're using an outdated API Version, please consider updating (server 1.6.12 / client 1.5.16)
fileName                        date                 description                                                                                                                                                           status    publicScore  privateScore  
------------------------------  -------------------  --------------------------------------------------------------------------------------------------------------------------------------------------------------------  --------  -----------  ------------  
kaggle_submission_ensemble.csv  2024-04-22 22:32:39  Ensemble Method num_prediction_vectors = 8                                                                                                                            pending                              
kaggle_submission_ensemble.csv